In [10]:
# Robotics control in a simulated environment
# Robotics Engineering
# 
# @file rt2_assignment_jupyter.ipynb
# @author Simone Contorno (@simone-contorno)
# @copyright Copyright (c) 2022

# Jupyter headers
import jupyros as jr
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox
from IPython.display import display, clear_output

# ROS headers
import rospy
import actionlib
from geometry_msgs.msg import Twist
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal

pub_goal = rospy.Publisher('/move_base/goal', MoveBaseActionGoal, queue_size = 1000)
#client = actionlib.SimpleActionClient('move_base', MoveBaseAction)
#client.wait_for_server()
send_out = widgets.Output()
canc_out = widgets.Output()
x = 0.0
y = 0.0
goal_flag = 0
flag = 0
rospy.set_param('/goal_flag', goal_flag)

# Button attributes
button_layout = Layout(width = '300px')
button_style = dict(
    font_style = 'italic',
    font_weight = 'bold',
    font_variant = "small-caps",
    text_color = 'red',
    text_decoration = 'none'
)

# Clear the button outputs
def clear_out(arg):
    send_out.clear_output()
    canc_out.clear_output()
    
# Go back to the interface
def back(arg):
    clear_output() # Clear the cell output
    interface()
    
# Terminate the program
def terminate(arg):
    #client.wait_for_server()
    rospy.set_param('/goal_flag', 0)
    #client.cancel_goal()
    
    clear_output() # Clear the cell output
    print("Bye.")

# Enable driving assistance
def enable_driving_assistance(arg):
    global flag
    flag = 1
    rospy.set_param('/drive_flag', 1)
    clear_output() # Clear the cell output
    interface()

# Disable driving assistance
def disable_driving_assistance(arg):
    global flag
    flag = 0
    rospy.set_param('/drive_flag', 0)
    clear_output() # Clear the cell output
    interface()

# Cancel the current goal
def cancel_goal(arg):
    goal_flag = rospy.get_param("/goal_flag")
    if goal_flag == 1:
        rospy.set_param('/goal_flag', 0)
        #client.cancel_goal()
        goal_flag = 0
        
        with canc_out:    
            canc_out.clear_output()
            print("Goal cancelled.\n")
    else: 
        with canc_out:    
            canc_out.clear_output()
            print("There is no goal set.\n")

# Send the new coordinates
def send_coordinates(arg):
    '''
    goal_pos = MoveBaseGoal()
    goal_pos.target_pose.header.frame_id = "map"
    goal_pos.target_pose.pose.orientation.w = 1
    goal_pos.target_pose.pose.position.x = x
    goal_pos.target_pose.pose.position.y = y
    '''
    goal_pos = MoveBaseActionGoal()    
    goal_pos.goal.target_pose.header.frame_id = "map"
    goal_pos.goal.target_pose.pose.orientation.w = 1;
    goal_pos.goal.target_pose.pose.position.x = x.value;
    goal_pos.goal.target_pose.pose.position.y = y.value;

    # Publish
    #client.wait_for_server()
    #client.send_goal(goal_pos)
    pub_goal.publish(goal_pos)
    rospy.set_param('/goal_flag', 1)
    
    with send_out:    
        send_out.clear_output()
        print("Goal sent.")

# Set new coordinated
def set_coordinates(arg):
    clear_output() # Clear the cell output
    print("Insert coordinates:")
    
    send_b = widgets.Button(description = 'Send', layout = button_layout, style = button_style) 
    send_b.style.button_color = 'lightgreen'
    back_b = widgets.Button(description = 'Back', layout = button_layout, style = button_style)
    back_b.style.button_color = 'red'
    
    global x
    global y 
    
    x = widgets.BoundedFloatText(
    value = 0.0,
    min = -100,
    step = 0.1,
    description = 'x :',
    disabled = False
    )
    y = widgets.BoundedFloatText(
    value = 0.0,
    min = -100,
    step = 0.1,
    description = 'y :',
    disabled = False
    )
    
    back_b.on_click(back)
    send_b.on_click(send_coordinates)
    
    display(x, y, send_b, back_b, send_out)


# Show the UI
def interface():
    global flag

    # Choose the action to perform
    print("\033[1;31m" + "\nWelcome to the User Interface!\n")
    print("\033[1;30m" + "Here you can choose between two different modalities to control your robot:\n" 
          "- automatic goal reaching, or\n"
          "- manual driving (with or without the driving assistance!)\n"
          "Choose an action:")
    
    exit_b = widgets.Button(description = 'Exit', layout = button_layout, style = button_style)
    exit_b.style.button_color = 'red'
    
    goal_b = widgets.Button(description = 'Insert new coordinates to reach', layout = button_layout, style = button_style)
    goal_b.style.button_color = 'lightgreen'
    
    canc_b = widgets.Button(description = 'Cancel the current goal', layout = button_layout, style = button_style)
    canc_b.style.button_color = 'lightgreen'

    drive_b = widgets.Button(description = 'Manual driving', layout = button_layout, style = button_style)
    drive_b.style.button_color = 'lightgreen'
    
    with_assist_b = widgets.Button(description = 'Enable driving assistance', layout = button_layout, style = button_style)
    with_assist_b.style.button_color = 'lightgreen'
    
    without_assist_b = widgets.Button(description = 'Disable driving assistance', layout = button_layout, style = button_style)
    without_assist_b.style.button_color = 'orange'
    
    clear_b = widgets.Button(description = 'Clear button output', layout = button_layout, style = button_style)
    clear_b.style.button_color = 'yellow'
    
    exit_b.on_click(terminate)
    goal_b.on_click(set_coordinates)
    canc_b.on_click(cancel_goal)
    #drive_b.on_click()
    with_assist_b.on_click(enable_driving_assistance)
    without_assist_b.on_click(disable_driving_assistance)
    clear_b.on_click(clear_out)
    
    if flag == 0:
        display(exit_b, goal_b, canc_b, canc_out, drive_b, with_assist_b, clear_b)   
    elif flag == 1:
        display(exit_b, goal_b, canc_b, canc_out, drive_b, without_assist_b, clear_b)
        
    '''
    # Manual driving
    elif res == '3':
        if goal_flag == 1:
            rospy.set_param('/goal_flag', 0)
            client.cancel_goal()
        (flag, counter1) = manualDriving(flag)
    '''
    
def main():
    rospy.init_node("jupyter_interface")    
    interface()

if __name__ == '__main__':
    main()


Welcome to the User Interface!

Here you can choose between two different modalities to control your robot:
- automatic goal reaching, or
- manual driving (with or without the driving assistance!)
Choose an action:


Button(description='Exit', layout=Layout(width='300px'), style=ButtonStyle(button_color='red', font_weight='bo…

Button(description='Insert new coordinates to reach', layout=Layout(width='300px'), style=ButtonStyle(button_c…

Button(description='Cancel the current goal', layout=Layout(width='300px'), style=ButtonStyle(button_color='li…

Output(outputs=({'output_type': 'stream', 'text': 'There is no goal set.\n\n', 'name': 'stdout'},))

Button(description='Manual driving', layout=Layout(width='300px'), style=ButtonStyle(button_color='lightgreen'…

Button(description='Enable driving assistance', layout=Layout(width='300px'), style=ButtonStyle(button_color='…

Button(description='Clear button output', layout=Layout(width='300px'), style=ButtonStyle(button_color='yellow…

In [ ]:
# Robotics control in a simulated environment
# Robotics Engineering
# 
# @file rt2_assignment_jupyter.ipynb
# @author Simone Contorno (@simone-contorno)
# @copyright Copyright (c) 2022

# Jupyter headers
import os
from jupyros import ros3d

view = ros3d.Viewer() # ROS 3D object
rc = ros3d.ROSConnection(url = "ws://localhost:9090") # Websocket (9090:rosbridge)
tf_client = ros3d.TFClient(ros = rc, fixed_frame = 'map') # Client connection 

# Visualize the map, the laser and the path to follow
map_view = ros3d.OccupancyGrid(topic = "/map", ros = rc, tf_client = tf_client) 
laser_view = ros3d.LaserScan(topic = "/scan", ros = rc, tf_client = tf_client) 
path = ros3d.Path(topic = "/move_base/NavfnROS/plan", ros = rc, tf_client = tf_client)

urdf = ros3d.URDFModel(ros = rc, tf_client = tf_client,
    path = os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))

g = ros3d.GridModel() # Grid object

# Show the model
view.objects = [g, laser_view, map_view, path, urdf]
view